In [28]:
import numpy as np
import pickle
import json
import copy
import pandas as pd
from tqdm import tqdm
import random

from perturbations import *
from helpers import *

# Read in data

Vocab

In [29]:
from Vocab import Vocab
vocab = Vocab('data/vocab.csv', 3)

Initialising vocab from file.


Segment data

In [30]:
from SegmentDataset import SegmentDataset

segment_test_set = SegmentDataset(
    data_dir='data/',
    segment_file='segments.json',
    vectors_file='vectors.json',
    split='test'
)

print("Testing the SegmentDataset class item getter...")
print("Dataset contains {} segment samples".format(len(segment_test_set)))
sample_id = 2
sample = segment_test_set[sample_id]
print("Segment {}:".format(sample_id))
print("Image set: {}".format(sample["image_set"]))
print("Target image index(es): {}".format(sample["targets"]))
# print("Target image Features: {}".format([segment_test_set.image_features[sample["image_set"][int(target)]] for target in sample["targets"]]))
print("Encoded segment: {}".format(sample["segment"]))
print("Decoded segment dialogue: {}".format(vocab.decode(sample["segment"])))
print("Segment length: ", sample["length"])
print("\nDone.")

Testing the SegmentDataset class item getter...
Dataset contains 6801 segment samples
Segment 2:
Image set: ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757']
Target image index(es): [6, 2, 3]
Encoded segment: [4, 21, 11, 757, 18, 6, 26, 49, 10, 5, 41]
Decoded segment dialogue: ['-A-', 'one', 'with', 'roses', 'and', 'a', 'white', 'laptop', '?', '-B-', 'nope']
Segment length:  11

Done.


Chain data

In [31]:
from ChainDataset import ChainDataset

print("Testing the ChainDataset class initialization...")

chain_test_set = ChainDataset(
    data_dir='data/',
    segment_file='segments.json',
    chain_file='test_chains.json',
    vectors_file='vectors.json',
    split='test'
)

print("Dataset contains {} cains.".format(len(chain_test_set.chains)))

sample_id = 5
sample = chain_test_set.chains[sample_id]

print("Chain {}:".format(sample_id))
print("Source Game ID: {}".format(sample["game_id"]))
print("Target image index: {}".format(sample["target"]))
print("Chain length: {}".format(len(sample["segments"])))
print("Segment IDs: {}".format(sample["segments"]))
print("Segment lengths: ", sample["lengths"])
print("First segment encoding: {}".format(chain_test_set.segments[sample["segments"][0]]["segment"]))
print("First segment decoded dialogue: {}".format(vocab.decode(chain_test_set.segments[sample["segments"][0]]["segment"])))
print("Second segment decoded dialogue: {}".format(vocab.decode(chain_test_set.segments[sample["segments"][1]]["segment"])))
# print("third segment decoded dialogue: {}".format(vocab.decode(chain_test_set.segments[sample["segments"][2]]["segment"])))

print("Reference chain and segments' associated image sets:")
for segment in sample["segments"]:
    print(vocab.decode(chain_test_set.segments[segment]["segment"]))
    print(chain_test_set.segments[segment]["image_set"])
print("\nDone.")

print("Testing the ChainDataset class item getter...")


Testing the ChainDataset class initialization...
Dataset contains 2811 cains.
Chain 5:
Source Game ID: 3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO
Target image index: 522129
Chain length: 4
Segment IDs: [3, 10, 12, 15]
Segment lengths:  [23, 23, 21, 19]
First segment encoding: [5, 38, 191, 23, 127, 11, 6, 120, 23, 127, 15, 9, 314, 10, 4, 13, 5, 80, 14, 128, 68, 22, 10]
First segment decoded dialogue: ['-B-', 'two', 'plates', 'of', 'food', 'with', 'a', 'bowl', 'of', 'food', 'in', 'the', 'middle', '?', '-A-', 'no', '-B-', 'okay', '.', 'how', 'about', 'you', '?']
Second segment decoded dialogue: ['-A-', 'do', 'you', 'have', 'the', 'one', 'with', 'two', 'plates', 'and', 'the', 'salsa', 'in', 'the', 'middle', '?', '-B-', 'i', 'do', "n't", 'have', 'that', 'one']
Reference chain and segments' associated image sets:
['-B-', 'two', 'plates', 'of', 'food', 'with', 'a', 'bowl', 'of', 'food', 'in', 'the', 'middle', '?', '-A-', 'no', '-B-', 'okay', '.', 'how', 'about', 'you', '?']


Game ids

In [32]:
with open('data/test_ids.json') as json_file:
    game_ids = json.load(json_file)
game_ids

['...']

# Models

Load model

In [33]:
from get_predictions import get_predictions
from helpers import *

In [8]:
# Get the datasets for experiment split
dataset_pred_no_hist, dataset_pred_hist_cp = get_pred_datasets(split='test')

loaded model: No history
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
params. normalize=True, mask=True, weight=5.5, weighting=True, batchsize=1, breaking=False
Dataparams. data_dir=./data, segmentfile=segments.json, vectorfile=vectors.json, chains_file=chains.json
segment dataset with segments.json vectors.json test
history dataset with test_segments.json vectors.json test_chains.json test
processing test

Start prediction
predict no history
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35, 'preds': tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]]), 'loss': tensor(4.9204), 'ranks': [0]}
getting predicitons took 81.22280263900757 seconds


In [34]:

# pickle.dump( dataset_pred_no_hist, open( "dataset_pred_no_hist.p", "wb" ) )
# pickle.dump( dataset_pred_hist_cp, open( "dataset_pred_hist_cp.p", "wb" ) )
dataset_pred_no_hist = pickle.load(open( "dataset_pred_no_hist.p", "rb" ) )
dataset_pred_hist_cp = pickle.load(open( "dataset_pred_hist_cp.p", "rb" ) )

In [35]:
# First index
ind = 24
print('Endocing of this segment is', dataset_pred_no_hist[ind]['segment'])
print('Decoded segment is', vocab.decode(dataset_pred_no_hist[ind]['segment']))
print('Length of the segment is', dataset_pred_no_hist[ind]['length'])
print('Images in the round of this segment is', dataset_pred_no_hist[ind]['image_set'])
print('Target of this segment is', dataset_pred_no_hist[ind]['targets'])
print('Rank (e.g. first time or third time this image was referenced in this game)', dataset_pred_no_hist[ind]['ranks'])
print('Predictions of the model on the segment is', dataset_pred_no_hist[ind]['preds'])
print('Loss of the predictions', dataset_pred_no_hist[ind]['loss'])

print(len(dataset_pred_no_hist))

Endocing of this segment is [5, 38, 59, 415, 23, 249, 18, 690, 10, 4, 17]
Decoded segment is ['-B-', 'two', 'pink', 'bowls', 'of', 'rice', 'and', 'broccoli', '?', '-A-', 'yes']
Length of the segment is 11
Images in the round of this segment is ['132002', '310714', '100022', '492731', '356290', '395097', '144797', '86285']
Target of this segment is [7, 2]
Rank (e.g. first time or third time this image was referenced in this game) [1, 1]
Predictions of the model on the segment is tensor([[[0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]])
Loss of the predictions tensor(12.6820)
6801


## Sort segments into conditions

The following things have been taken into account when creating the dataframes:
- Only the predictions where there was only one target image given a segment (and thus not multiple target images given a segment)
- Only the predictions where the model predicts one image (and thus not multiple images given a segment)

correct_prediction = 1 when the model predicts correctly, 0 when the model predicts wrong

In [36]:
dataframe = get_pred_dataframe(dataset_pred_no_hist, dataset_pred_hist_cp)
    

 23%|█████████████████                                                          | 1546/6801 [00:00<00:00, 15340.49it/s]

[4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history


 70%|████████████████████████████████████████████████████▎                      | 4743/6801 [00:00<00:00, 23545.02it/s]

History


100%|███████████████████████████████████████████████████████████████████████████| 6801/6801 [00:00<00:00, 22118.06it/s]


Sort the segments in their conditions (whether the history/no history dataset predicted them correctly)

hT_nhF - history True, no-history False (so it was correctly predicted with history, incorrectly without)

Conditions_inds is a dictionary of lists, {'condition': [segment indices for this condition]}

In [37]:
conditions_inds = get_conditions_inds(dataframe)

print("Number of segments in conditions")
counter = 0
for key in conditions_inds:
    print(key, len(conditions_inds[key]))
    counter +=len(conditions_inds[key])
counter

Number of segments in conditions
hT_nhT 1702
hT_nhF 112
hF_nhT 121
hF_nhF 248
only_h 681
only_nh 787
nothing 3150


6801

In [38]:
print("The number of segments per condition that have a history:")
# For each condition
for condition in conditions_inds:
    counter = 0
#     for seg_id in range(len(dataset_pred_hist_cp)):
    # Iterate over the indices in this condition
    for seg_id in conditions_inds[condition]:
        # For each chain that this segment belongs to
        for chain_i in range(len(dataset_pred_hist_cp[seg_id]['chains'])):
            # Add counter if the segment is not the first round
            if dataset_pred_hist_cp[seg_id]['rounds'][chain_i] > 0:
                counter += 1
                break
    print(condition, counter)

The number of segments per condition that have a history:
hT_nhT 1144
hT_nhF 84
hF_nhT 61
hF_nhF 154
only_h 419
only_nh 548
nothing 2439


In [40]:
results_hist, results_nohist, accs_hist, accs_nohist = get_accuracies(conditions_inds, dataframe)
print("history accuracy")
print(accs_hist)
print("no history accuracy")
print(accs_nohist)


history accuracy
{'hT_nhT': 1.0, 'hT_nhF': 1.0, 'hF_nhT': 0.0, 'hF_nhF': 0.0, 'only_h': 0.7430249632892805, 'only_nh': nan, 'nothing': nan}
no history accuracy
{'hT_nhT': 1.0, 'hT_nhF': 0.0, 'hF_nhT': 1.0, 'hF_nhF': 0.0, 'only_h': nan, 'only_nh': 0.7306226175349428, 'nothing': nan}


## Combine segments with first in the chain for analyses
Create a dict with the first segment and current segment of a condition


In [41]:
condition_seg_hist = get_condition_seg_hist(conditions_inds, dataset_pred_hist_cp)

In [42]:
seg_id = 11
condition_seg_hist['hT_nhT'][13]

{2: {'first_id': 2,
  'first_seg': [4, 21, 11, 757, 18, 6, 26, 49, 10, 5, 41],
  'current_id': 13,
  'current_seg': [5, 549, 23, 435, 51, 29, 26, 49, 10, 4, 13, 5, 80],
  'round': 1}}

TODO: add analyses here?

## Perturbations

If we make a new perturbation:
    
    * create a new segment and chain file, even if one of them is the same as the test file. Leave vocab file just as it is.
    * Make sure the file format is split+_chains.json and split+segments.json, where split is e.g. 'test_shuffle'. Split can be used to indicate the experiment that is run
    * Also let these files be put into the '/data' folder
    * When shuffeling files based on chain_test_set or segments_test set, make a deep copy first!!
    * If you change the segments in the chains json file, make sure you also update the lengths
    
    * add the experiment and the correspnding segment and chain files in the experiment_names dictionary
  

##### extra info
The training model takes these files as input. 

Change chains if we want to switch up the segments (i.e. from other games or change the order)

Change segments if we want to change things within the utterances

In [43]:
# For later analyses
experiment_names = {"standard":{'segments_file':'data/test_chains.json',
                                'chains_file':'data/test_segments.json',
                                'split':'test'},
                   "shuffle":{'segments_file':'data/test_shuffle_chains.json',
                                'chains_file':'data/test_shuffle_segments.json',
                                'split':'test_shuffle'},
                    
                    "pos_tag_noun":{'segments_file':'data/test_noun_segments.json',
                                'chains_file':'data/test_noun_chains.json',
                                'split':'test_noun'},
                    "pos_tag_verb":{'segments_file':'data/test_verb_segments.json',
                                'chains_file':'data/test_verb_chains.json',
                                'split':'test_verb'},
                    "pos_tag_adj":{'segments_file':'data/test_adj_segments.json',
                                'chains_file':'data/test_adj_chains.json',
                                'split':'test_adj'},
                    "pos_tag_adv":{'segments_file':'data/test_adv_segments.json',
                                'chains_file':'data/test_adv_chains.json',
                                'split':'test_adv'},
                    
                   "games_0":{'segments_file':'data/test_games_0_segments.json',
                                'chains_file':'data/test_games_0_chains.json',
                                'split':'test_games_0'},
                   "games_1":{'segments_file':'data/test_games_1_segments.json',
                                'chains_file':'data/test_games_1_chains.json',
                                'split':'test_games_1'},
                   "games_2":{'segments_file':'data/test_games_2_segments.json',
                                'chains_file':'data/test_games_2_chains.json',
                                'split':'test_games_2'},
                   "games_3":{'segments_file':'data/test_games_3_segments.json',
                                'chains_file':'data/test_games_3_chains.json',
                                'split':'test_games_3'},
                   "games_4":{'segments_file':'data/test_games_4_segments.json',
                                'chains_file':'data/test_games_4_chains.json',
                                'split':'test_games_4'},
                    
                   "imgs_0":{'segments_file':'data/test_imgs_0_segments.json',
                                'chains_file':'data/test_imgs_0_chains.json',
                                'split':'test_imgs_0'},   
                   "imgs_1":{'segments_file':'data/test_imgs_1_segments.json',
                                'chains_file':'data/test_imgs_1_chains.json',
                                'split':'test_imgs_1'},   
                   "imgs_2":{'segments_file':'data/test_imgs_2_segments.json',
                                'chains_file':'data/test_imgs_2_chains.json',
                                'split':'test_imgs_2'},   
                   "imgs_3":{'segments_file':'data/test_imgs_3_segments.json',
                                'chains_file':'data/test_imgs_3_chains.json',
                                'split':'test_imgs_3'},   
                   "imgs_4":{'segments_file':'data/test_imgs_4_segments.json',
                                'chains_file':'data/test_imgs_4_chains.json',
                                'split':'test_imgs_4'},   
                   }

for experiment in experiment_names:
    experiment_names[experiment]['accs_hist'] = []
    experiment_names[experiment]['accs_nohist'] = []

In [44]:
experiment_names

{'standard': {'segments_file': 'data/test_chains.json',
  'chains_file': 'data/test_segments.json',
  'split': 'test',
  'accs_hist': [],
  'accs_nohist': []},
 'shuffle': {'segments_file': 'data/test_shuffle_chains.json',
  'chains_file': 'data/test_shuffle_segments.json',
  'split': 'test_shuffle',
  'accs_hist': [],
  'accs_nohist': []},
 'pos_tag_noun': {'segments_file': 'data/test_noun_segments.json',
  'chains_file': 'data/test_noun_chains.json',
  'split': 'test_noun',
  'accs_hist': [],
  'accs_nohist': []},
 'pos_tag_verb': {'segments_file': 'data/test_verb_segments.json',
  'chains_file': 'data/test_verb_chains.json',
  'split': 'test_verb',
  'accs_hist': [],
  'accs_nohist': []},
 'pos_tag_adj': {'segments_file': 'data/test_adj_segments.json',
  'chains_file': 'data/test_adj_chains.json',
  'split': 'test_adj',
  'accs_hist': [],
  'accs_nohist': []},
 'pos_tag_adv': {'segments_file': 'data/test_adv_segments.json',
  'chains_file': 'data/test_adv_chains.json',
  'split': 't

### Shuffle experiment
Change order of the segments in a chain

In [45]:
import copy
from perturbations import chain_shuffle

# Get a deepcopy of the chain list
chain_test_shuf_ch = copy.deepcopy(chain_test_set.chains)   

# Shuffle segment order
# Make sure output file ends with chains.json. So now 'test_shuf' is the split argument in following functions
data_chain_shuf, data_seg_shuf = chain_shuffle(chain_test_shuf_ch, chain_file='data/test_shuffle_chains.json', segment_file='data/test_shuffle_segments.json')

In [46]:
pert_sanity_check(test_chains_exp='data/test_shuffle_chains.json', test_segm_exp='data/test_shuffle_segments.json')

Sanity check
original chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [0, 7, 8], 'lengths': [35, 15, 33]}
experiment chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [7, 0, 8], 'lengths': [15, 35, 33]}

original segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
experiment segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
-----------------------------------
origin

## Game exchange experiment
For a certain round round_id, change the segment in that round for a segment on the same image from the same round but a random game

# TODO: create function that creates file and then does experiment (also taking into account changed_seg_ids)

In [47]:
for round_id in [0, 1, 2, 3, 4]:
    chain_copy = copy.deepcopy(chain_test_set.chains)   
    chain_test_game_n, segments, changed_seg_ids = pert_exchange_games(chain_copy, round_id, chain_file_start='data/test_games_', segment_file_start='data/test_games_', output=True)


In [48]:
pert_sanity_check(test_chains_exp='data/test_games_1_chains.json', test_segm_exp='data/test_games_1_segments.json')

Sanity check
original chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [0, 7, 8], 'lengths': [35, 15, 33]}
experiment chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [0, 4977, 8], 'lengths': [35, 35, 33]}

original segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
experiment segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
-----------------------------------
ori

## Image exchange experiment
For a round_id, change the segment for another segment on another target image but from the same game

In [53]:

# def pert_exchange_imgs(chain_copy, round_id, chain_file_start='data/test_imgs_', segment_file_start='data/test_imgs_', output=True):
#     """ 
#     Chain_copy is a deepcopy of chain_test_set.chains
#     """
    
#     chain_test_game_n, changed_seg_ids = exchange_images(chain_copy, round_id)
    
#     # Write new chains file
#     if output:
#         with open(chain_file_start+str(round_id)+'_chains.json', 'w') as json_file:
#             json.dump(chain_test_game_n, json_file)
#         # Keep segment file the same
#         with open('data/test_segments.json') as json_file:
#             segments = json.load(json_file)
#         with open(segment_file_start+str(round_id)+'_segments.json', 'w') as json_file:
#             json.dump(segments, json_file)
#     return chain_test_game_n, segments, changed_seg_ids


# def exchange_images(chains, round_id):
#     """
#     chains is chain_test_set.chains
#     round_id is the round that we want to change
    
#     Returns the new chain test set and the segment indices of the new segments in the chains.
#     """
#     # Get a deepcopy of the chain list
#     chain_test_game_n = copy.deepcopy(chains)   
#     game_dict = get_game_dict(chains)
#     changed_seg_ids = []
    
#     # Shuffle the target indices
#     for game_id in game_dict:
#         if round_id in game_dict[game_id]:
#             random.shuffle(game_dict[game_id][round_id]['chain_ids_shuf'])

#     # Iterate over the indices in the chain set
#     for chain_id in range(len(chain_test_game_n)):
#         game_id = chain_test_game_n[chain_id]['game_id']

#         # Check if there are at least round_id rounds in this chain
#         if len(chain_test_game_n[chain_id]['segments']) <= round_id:
#             continue
#         # Check if there are other games
#         if len(set(game_dict[game_id][round_id]['targets'])) == 1:
#             continue

#         # Get index of the new segment, given the shuffled chain ids
#         new_seg_ind = game_dict[game_id][round_id]['chain_ids_shuf'].index(chain_id)

#         # Change segment in round_id to the new_seg_id
#         chain_test_game_n[chain_id]['segments'][round_id] = game_dict[game_id][round_id]['segments'][new_seg_ind]
#         chain_test_game_n[chain_id]['lengths'][round_id] = game_dict[game_id][round_id]['lengths'][new_seg_ind]

#         # Save new segment id for analysis
#         changed_seg_ids.append(game_dict[game_id][round_id]['segments'][new_seg_ind])
#     return chain_test_game_n, changed_seg_ids


# def get_game_dict(chains):
#     """
#     chains is chain_test_set.chains
#     Return a dictionary of
#     {game_id:{'round_id':'segments':[segids],
#                         'lengths':[seglengths],
#                         'game_ids':[gameids]}}
#     where game_id is the game_id of the chain
#     round_id is the round/rank, i.e. the i'th time the image is being talked about
#     segid is the segment id
#     seglength is the length of the segment with the corresponding index
#     gameid is the gameid to which the segment with the corresponding index belongs

#     Used for exchange_images()
#     """
#     game_dict = {}
#     # Get segments, lengths and gameids
#     for chain_id in range(len(chains)):
#         image_ind = chains[chain_id]['target']
#         game_id = chains[chain_id]['game_id']
#         # Add gameid to the game_dict    
#         if game_id not in game_dict:
#             game_dict[game_id] = {}

#         # Iterate over the rounds/segments
#         for round_ind in range(len(chains[chain_id]['segments'])):

#             # Add round to the dict for this target image
#             if round_ind not in game_dict[game_id]:
#                 game_dict[game_id][round_ind] = {'segments':[],
#                                                   'lengths':[],
#                                                  'game_ids':[],
#                                                  'targets':[],
#                                                  'chain_ids':[],
#                                                  'chain_ids_shuf':[]}

#             # Add segment with the length and gameid info to this round dict
#             game_dict[game_id][round_ind]['segments'].append(chains[chain_id]['segments'][round_ind])
#             game_dict[game_id][round_ind]['lengths'].append(chains[chain_id]['lengths'][round_ind])
#             game_dict[game_id][round_ind]['game_ids'].append(game_id)
#             game_dict[game_id][round_ind]['targets'].append(image_ind)
#             game_dict[game_id][round_ind]['chain_ids'].append(chain_id)
#             game_dict[game_id][round_ind]['chain_ids_shuf'].append(chain_id)

#     return game_dict





In [54]:
for round_id in [0, 1, 2, 3, 4]:
    chain_copy = copy.deepcopy(chain_test_set.chains)   
    chain_test_game_n, segments, changed_seg_ids = pert_exchange_imgs(chain_copy, round_id, chain_file_start='data/test_imgs_', segment_file_start='data/test_imgs_', output=True)


In [58]:
pert_sanity_check(test_chains_exp='data/test_imgs_1_chains.json', test_segm_exp='data/test_imgs_1_segments.json')

Sanity check
original chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [0, 7, 8], 'lengths': [35, 15, 33]}
experiment chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [0, 6, 8], 'lengths': [35, 20, 33]}

original segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
experiment segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
-----------------------------------
origin

## Run perturbation predictions
Get the accuracies per condition for an experiment
# TODO: create a function with flags to use one of the specific experiment functions

In [24]:
def experiment_accs(conditions_inds, split='test'):
    # Get dataframes
    dataset_pred_no_hist_exp, dataset_pred_hist_cp_exp = get_pred_datasets(split=split)
    # Get predictions of the segments
    dataframe_exp = get_pred_dataframe(dataset_pred_no_hist_exp, dataset_pred_hist_cp_exp)
    # Get the accuracy dict
    results_hist, results_nohist, accs_hist, accs_nohist = get_accuracies(conditions_inds, dataframe_exp)
    
    return accs_hist, accs_nohist, dataframe_exp, dataset_pred_hist_cp_exp, dataset_pred_no_hist_exp
    

In [25]:
def run_experiment(experiment):

    split = experiment_names[experiment]['split']
    accs_hist, accs_nohist, dataframe_exp, dataset_pred_hist_cp_exp, dataset_pred_no_hist_exp = experiment_accs(conditions_inds, split=split)

    print(f"Experiment{experiment} accuracies:")
    print("history:")
    print(accs_hist)
    print("No history")
    print(accs_nohist)

    return accs_hist, accs_nohist

In [26]:
experiment = 'games_0'
accs_hist, accs_nohist = run_experiment(experiment)

loaded model: No history
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
params. normalize=True, mask=True, weight=5.5, weighting=True, batchsize=1, breaking=False
Dataparams. data_dir=./data, segmentfile=segments.json, vectorfile=vectors.json, chains_file=chains.json
segment dataset with segments.json vectors.json test_games_0
history dataset with test_games_0_segments.json vectors.json test_games_0_chains.json test_games_0
processing test_games_0

Start prediction
predict no history
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35, 'preds': tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]]), 'loss': tensor(4.9204), 'ranks': [0]}
getting pre

 34%|█████████████████████████▋                                                 | 2324/6801 [00:00<00:00, 23023.15it/s]

[4, 8, 142, 71, 14, 110, 22, 10, 5, 13, 12, 22, 7, 6, 49, 11, 294, 18, 2049] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history


 39%|█████████████████████████████▎                                             | 2658/6801 [00:00<00:00, 26393.88it/s]

History


100%|███████████████████████████████████████████████████████████████████████████| 6801/6801 [00:00<00:00, 24356.38it/s]


Experimentgames_0 accuracies:
history:
{'hT_nhT': 0.8018617021276596, 'hT_nhF': 0.7560975609756098, 'hF_nhT': 0.8, 'hF_nhF': 0.8288288288288288, 'only_h': 0.7306273062730627, 'only_nh': 0.7863501483679525, 'nothing': 0.7773695811903012}
No history
{'hT_nhT': 1.0, 'hT_nhF': 0.0, 'hF_nhT': 1.0, 'hF_nhF': 0.0, 'only_h': nan, 'only_nh': 0.7306226175349428, 'nothing': nan}


In [27]:
experiment = 'games_1'
accs_hist, accs_nohist = run_experiment(experiment)

loaded model: No history
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
params. normalize=True, mask=True, weight=5.5, weighting=True, batchsize=1, breaking=False
Dataparams. data_dir=./data, segmentfile=segments.json, vectorfile=vectors.json, chains_file=chains.json
segment dataset with segments.json vectors.json test_games_1
history dataset with test_games_1_segments.json vectors.json test_games_1_chains.json test_games_1
processing test_games_1

Start prediction
predict no history
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35, 'preds': tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]]), 'loss': tensor(4.9204), 'ranks': [0]}
getting pre

 35%|██████████████████████████                                                 | 2366/6801 [00:00<00:00, 23483.77it/s]

[4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history


 40%|██████████████████████████████▎                                            | 2754/6801 [00:00<00:00, 27340.37it/s]

History


100%|███████████████████████████████████████████████████████████████████████████| 6801/6801 [00:00<00:00, 23677.81it/s]

Experimentgames_1 accuracies:
history:
{'hT_nhT': 0.7699859747545582, 'hT_nhF': 0.7884615384615384, 'hF_nhT': 0.7948717948717948, 'hF_nhF': 0.7184466019417476, 'only_h': 0.7828947368421053, 'only_nh': 0.7889908256880734, 'nothing': 0.7894736842105263}
No history
{'hT_nhT': 1.0, 'hT_nhF': 0.0, 'hF_nhT': 1.0, 'hF_nhF': 0.0, 'only_h': nan, 'only_nh': 0.7306226175349428, 'nothing': nan}


Perform experiments multiple times

In [ ]:
standard_results = []
shuffle_results = []
pos_tag_noun_results = []
pos_tag_verb_results = []
pos_tag_adj_results = []
pos_tag_adv_results = []

# Perform experiments multiple times
for experiment in range(1):

    for key, value in experiment_names.items():

        if key == "standard":
            history_result, no_history_result = run_experiment(key)
            standard_results.append(history_result, no_history_result)
        if key == "shuffle":
            history_result, no_history_result = run_experiment(key)
            shuflle_results.append(history_result, no_history_result)

        if key == "pos_tag_noun":
            history_result, no_history_result = run_experiment(key)
            pos_tag_noun_results.append(history_result, no_history_result)

        if key == "pos_tag_verb":
            history_result, no_history_result = run_experiment(key)
            pos_tag_verb_results.append(history_result, no_history_result)

        if key == "pos_tag_adj":
            history_result, no_history_result = run_experiment(key)
            pos_tag_adj_results.append(history_result, no_history_result)     

        if key == "pos_tag_adv":
            history_result, no_history_result = run_experiment(key)
            pos_tag_adv_results.append(history_result, no_history_result)
            



    
    